## Proyecto Music Stream


### Objetivo

1. Crear una bases de datos para almacenar información de canciones y artistas. 
2. Extraer información de la base de datos para responder preguntas sobre la música almacenada.

### Fases del proyecto 

**Fase 1: Extraer Datos**
1. Extraer información de Api Spotify
2. Extraer información de API Last FM
3. Guardar esa información en CSV

**Fase 2:Bases de datos**
1. Diseñar la Base de Datos
2. Crear la bases de datos
3. Insertar los valores en la bases de datos
   
**Fase 3:Extraer información**
1. Extraer información de la base de datos para responder preguntas sobre la música almacenada.
   

## Comencemos... Fase 1.1 Extraer información de Api Spotify

- Debemos usar la librería spotipy para extraer información de la API de Spotify: https://spotipy.readthedocs.io/en/2.24.0/ 
- Debemos hacer una cuenta en Spotify para obtener las credenciales necesarias para usar la API de Spotify.
- `!pip install spotipy`



In [ ]:
!pip install spotipy

In [ ]:
import spotipy

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = '04f685005f584087a8cd9fada4a373f5'
CLIENT_SECRET = '95e82b9914dc42e2a82ab0132df82dfb'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

### Entendiendo el API de Spotipy 


- La documentación del API Spotify, el método que vamos a usar: https://spotipy.readthedocs.io/en/2.24.0/index.html#spotipy.client.Spotify.search


Tenemos disponible el método `search(q, limit=10, offset=0, type='track', market=None)` donde: 

- `search` busca un artículo que cumpla las condiciones de la búsqueda, que van ser: 
- q: la cadena de búsqueda limitando la búsqueda utilizando filtros de campo. Los filtros disponibles son `álbum`, `artista`, `pista`, `año`, `upc`, `etiqueta:hipster`, `etiqueta:nuevo`, isrc y `género`. Cada filtro de campo solo se aplica a ciertos tipos de resultados.
  - Los filtros de artista y año se pueden utilizar al buscar álbumes, artistas y pistas. Puede filtrar por un solo año o por un rango (por ejemplo, 1955-1960).
  - El filtro de álbumes se puede utilizar mientras se buscan álbumes y pistas
  - El filtro de género se puede utilizar al buscar artistas y pistas.
  - Los filtros isrc y track se pueden utilizar mientras se buscan pistas.
  - Los filtros upc, tag:new y tag:hipster solo se pueden utilizar durante la búsqueda de álbumes.
  -  El filtro etiqueta:nuevo devolverá álbumes lanzados en las últimas dos semanas y etiqueta:hipster se puede utilizar para devolver sólo álbumes con el 10% de popularidad más bajo. 

- limit: el número máximo de elementos que se devolverán. El valor predeterminado es 10 y el límite máximo es 50.
- tipo: Una lista separada por comas de tipos de elementos para buscar. Los resultados de la búsqueda incluyen visitas de todos los tipos de elementos especificados. Por ejemplo: q=abacab&type=album,track devuelve álbumes y pistas que coinciden con "abacab". Valores permitidos: "album", "artist", "playlist", "track", "show", "episode", "audiobook"


- Aquí podemos obtener datos como artista, genero, tipo (canción o albuum), nombre, año de lanzamiento, 

  


### Como usar el método de search de Spotipy

In [ ]:
#Estamos buscando las canciones que sean de genero rock del 2020, la consulta lo ponemos en la query
datos = sp.search(q="genre:rock year:2020", type='track', limit=50, offset=0)
data_track_album = sp.search(q="genre:rock year:2020", type='track,album', limit=50, offset=0) #para ambos

In [ ]:
#aqui vemos que me trae como respuesta, y vemos en el resultado qeu viene dentro de tracks
datos.keys()

In [ ]:
datos

In [ ]:

# Aqui veo la info que viene en tracks datos[`tracks`], pero no es suficiente, al parecer la información viene en items, entonces hago un datos['tracks']['items'], como me sigue sin ver bien solo voy a ver un elemento a ver como se ve y hago datos['tracks']['items'][0]
datos['tracks']['items'][0]

In [ ]:
track_only_one = datos['tracks']['items'][2]

In [ ]:
track_only_one

In [ ]:

track_object = {
    'name_track': track_only_one['name'],
    'name_artist': track_only_one['artists'][0]['name'],
    'genre': "rock", #no lo devuelve, lo tengo que poner según el género que esté buscando
    'type': track_only_one['type'], 
    'year': track_only_one['album']['release_date'][:4] 
}
track_object

## ahora esto solo está hecho para una canción, de todo el resultado que me devuelve
#tendremos que hacer esto mismo para todo el listado: pista hay que hacer un for
# la información la guardaremos en un listado de objectos, donde cada objetos es la información de la canción

resultados_totales = []
for item in datos['tracks']['items']:
    resultados_totales.append({
             'name_track': item['name'],
            'name_artist': item['artists'][0]['name'],
            'genre': "rock", #no lo devuelve, lo tengo que poner según el género que esté buscando
            'type': item['type'], 
            'year': item['album']['release_date'][:4] })

In [ ]:
#para ver mi listado de canciones
resultados_totales

In [ ]:
for track in resultados_totales:
    print(track['name_track'])


## Proyecto Music Stream

## Seguimos... Fase 1.2 Extraer información de Api Last FM

- Hay que ir a la página de Last FM y registrarse para obtener una API KEY. https://www.last.fm/api 
- Consultar la documentación de la API de Last FM para saber como hacer las consultas.https://www.last.fm/api 
- La dirección URL de la API de Last FM es: http://ws.audioscrobbler.com/2.0/
- Aquí podemos obtener información por artista, como: biografia, oyentes, numero de reproducciones, artistas similares


In [63]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
#pip install mysql-connector -> en caso de no tenerla
import mysql.connector
from mysql.connector import errorcode

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
#pip install requests-> en caso de no tenerla
#pip install pandas-> en caso de no tenerla
#pip install numpy-> en caso de no tenerla
import requests
import pandas as pd

# conexion a spotify
# -----------------------------------------------------------------------
#pip install spotipy -> en caso de no tenerla
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [79]:
# Este script consulta los artistas en base a una lista que yo le paso (que traigo desde la consulta de spotify). Su informacion sera artista, 
# Biography, Listeners, Playcount, Similar Artists


# API Key obtenida de last.fm
API_KEY = '761de5f859b5836a8e26c24538344465'  # Reemplaza con tu API Key válida de Last.fm.
BASE_URL = 'https://ws.audioscrobbler.com/2.0/?method=auth.gettoken&api_key=761de5f859b5836a8e26c24538344465&format=json' # esta base siempre es la misma.

In [86]:
# Lista para almacenar los datos que luego guardaremos en un DataFrame
artist_data = []
params_info = {
        'method': 'artist.getinfo',
        'artist': "Madonna",
        'api_key': API_KEY,
        'format': 'json'
    }
response_info = requests.get(BASE_URL, params=params_info)
data_info = response_info.json()
data_info


{'artist': {'name': 'Madonna',
  'mbid': '79239441-bfd5-4981-a70c-55c3f15c1287',
  'url': 'https://www.last.fm/music/Madonna',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0',
  'ontour': '0',
  'stats': {'listeners': '4800057', 'playcount': '269364294'},
  'similar': {'artist': [{'name': 'Kylie Minogu

In [85]:
## Vamos a crear un objeto artista 

artista = {
    'Artist': "Madonna",
    'Biography': data_info['artist'].get('bio', {}).get('summary'),
    'Listeners': data_info['artist'].get('stats', {}).get('listeners', "Sin datos"),
    'Playcount': data_info['artist'].get('stats', {}).get('playcount', "Sin datos"),
    
}
artista


{'Artist': 'Madonna',
 'Biography': 'Madonna Louise Veronica Ciccone (born August 16, 1958 in Bay City, Michigan) is an American singer, songwriter and actress. She is considered one of the most influential figures in popular culture and has often been referred to as the "Queen of Pop". Madonna is noted for her continual reinvention and versatility in music production, songwriting and visual presentation. She is also known for pushing the boundaries of artistic expression in mainstream music, while maintaining control over every aspect of her career. <a href="https://www.last.fm/music/Madonna">Read more on Last.fm</a>',
 'Listeners': '4800057',
 'Playcount': '269364294'}

In [91]:
## Para obtener los artistas similares de un artisa
params_similar = {
        'method': 'artist.getsimilar',
        'artist': "Madonna",
        'api_key': API_KEY,
        'format': 'json'
    }

response_similar = requests.get(BASE_URL, params=params_similar)
data_similar = response_similar.json()
data_similar['similarartists']['artist']

[{'name': 'Kylie Minogue',
  'mbid': '2fddb92d-24b2-46a5-bf28-3aed46f4684c',
  'match': '1',
  'url': 'https://www.last.fm/music/Kylie+Minogue',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0'},
 {'name': 'Britney Spears',
  'mbid': '45a663b5-b1cb-4a91-bff6-2bef7bbfdd76',
  'match': '0.663564',
  'url'

In [93]:
# Hago un listado con los valores similares
similar_artists=[]
for artist in data_similar['similarartists']['artist']:
    similar_artists.append(artist['name'])
similar_artists


['Kylie Minogue',
 'Britney Spears',
 'Janet Jackson',
 'Paula Abdul',
 'Cher',
 'Dannii Minogue',
 'Christina Aguilera',
 'Lady Gaga',
 'Jennifer Lopez',
 'Mariah Carey',
 'Sophie Ellis-Bextor',
 'Donna Summer',
 'Spice Girls',
 'Cyndi Lauper',
 'Gwen Stefani',
 'Robyn',
 'Whitney Houston',
 'Taylor Dayne',
 'Katy Perry',
 'Rihanna',
 'Tina Turner',
 'Mylène Farmer',
 'Grace Jones',
 'Shakira',
 'Céline Dion',
 'Geri Halliwell',
 'Girls Aloud',
 'Jessie Ware',
 'Ke$ha',
 'Cathy Dennis',
 'George Michael',
 'Nelly Furtado',
 'Róisín Murphy',
 'Melanie C',
 'Fergie',
 'Shania Twain',
 'Paris Hilton',
 'Miley Cyrus',
 'Diana Ross',
 'Kate Bush',
 'Heidi Montag',
 'Michael Jackson',
 'Sugababes',
 'Crystal Waters',
 'Annie Lennox',
 'Tove Lo',
 'Bananarama',
 'Alanis Morissette',
 'Barbra Streisand',
 'Lindsay Lohan',
 'Goldfrapp',
 'Rachel Stevens',
 'Beyoncé',
 'The Pussycat Dolls',
 'RuPaul',
 'P!nk',
 'Wanessa',
 'Emma Bunton',
 'Jessica Simpson',
 'Kelis',
 'Gloria Estefan',
 'Hilary

In [78]:
artista['similar_artists'] = similar_artists
artista

{'Artist': 'Madonna',
 'Biography': 'Madonna Louise Veronica Ciccone (born August 16, 1958 in Bay City, Michigan) is an American singer, songwriter and actress. She is considered one of the most influential figures in popular culture and has often been referred to as the "Queen of Pop". Madonna is noted for her continual reinvention and versatility in music production, songwriting and visual presentation. She is also known for pushing the boundaries of artistic expression in mainstream music, while maintaining control over every aspect of her career. <a href="https://www.last.fm/music/Madonna">Read more on Last.fm</a>',
 'Listeners': '4798951',
 'Playcount': '269282771',
 'similar_artists': ['Kylie Minogue',
  'Britney Spears',
  'Janet Jackson',
  'Paula Abdul',
  'Cher',
  'Dannii Minogue',
  'Christina Aguilera',
  'Lady Gaga',
  'Jennifer Lopez',
  'Mariah Carey',
  'Sophie Ellis-Bextor',
  'Donna Summer',
  'Spice Girls',
  'Cyndi Lauper',
  'Gwen Stefani',
  'Robyn',
  'Whitney H

### Ejemplo de resultado:

- Una tabla con el nombre de artista, reproducciones, oyentes, biografia y artistas similares
- Una tabla de canciones: con el nombre de artista, nombre de cancion, nombre del album, año, genero, id y popularidad
